In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 7)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.407615065574646, Final Batch Loss: 1.1043334007263184
Epoch 2, Loss: 4.398714065551758, Final Batch Loss: 1.108474612236023
Epoch 3, Loss: 4.385259985923767, Final Batch Loss: 1.1003540754318237
Epoch 4, Loss: 4.363013863563538, Final Batch Loss: 1.0798442363739014
Epoch 5, Loss: 4.355208516120911, Final Batch Loss: 1.091536045074463
Epoch 6, Loss: 4.3286179304122925, Final Batch Loss: 1.079339623451233
Epoch 7, Loss: 4.304621458053589, Final Batch Loss: 1.0723742246627808
Epoch 8, Loss: 4.264464855194092, Final Batch Loss: 1.0609840154647827
Epoch 9, Loss: 4.220078706741333, Final Batch Loss: 1.0450940132141113
Epoch 10, Loss: 4.162905812263489, Final Batch Loss: 1.0374436378479004
Epoch 11, Loss: 4.086629152297974, Final Batch Loss: 1.0132759809494019
Epoch 12, Loss: 3.9892364144325256, Final Batch Loss: 0.9896432757377625
Epoch 13, Loss: 3.852113425731659, Final Batch Loss: 0.9519662857055664
Epoch 14, Loss: 3.6816746592521667, Final Batch Loss: 0.8905184864997864
E

Epoch 121, Loss: 0.20679648220539093, Final Batch Loss: 0.050010159611701965
Epoch 122, Loss: 0.26600281894207, Final Batch Loss: 0.08677597343921661
Epoch 123, Loss: 0.2739914432168007, Final Batch Loss: 0.05807482823729515
Epoch 124, Loss: 0.2560640834271908, Final Batch Loss: 0.08005407452583313
Epoch 125, Loss: 0.307220209389925, Final Batch Loss: 0.139631986618042
Epoch 126, Loss: 0.2643237076699734, Final Batch Loss: 0.06467802822589874
Epoch 127, Loss: 0.24748511612415314, Final Batch Loss: 0.06330756843090057
Epoch 128, Loss: 0.22050448320806026, Final Batch Loss: 0.022290444001555443
Epoch 129, Loss: 0.26227834820747375, Final Batch Loss: 0.054410867393016815
Epoch 130, Loss: 0.2891252972185612, Final Batch Loss: 0.10203540325164795
Epoch 131, Loss: 0.21887899935245514, Final Batch Loss: 0.04141683503985405
Epoch 132, Loss: 0.2642732486128807, Final Batch Loss: 0.04446975514292717
Epoch 133, Loss: 0.28763920441269875, Final Batch Loss: 0.1107063814997673
Epoch 134, Loss: 0.246

Epoch 233, Loss: 0.2255111914128065, Final Batch Loss: 0.0878181979060173
Epoch 234, Loss: 0.18177083134651184, Final Batch Loss: 0.07615618407726288
Epoch 235, Loss: 0.1527277585119009, Final Batch Loss: 0.028770051896572113
Epoch 236, Loss: 0.17594588734209538, Final Batch Loss: 0.05166855826973915
Epoch 237, Loss: 0.12012515496462584, Final Batch Loss: 0.010812350548803806
Epoch 238, Loss: 0.14767546765506268, Final Batch Loss: 0.059348151087760925
Epoch 239, Loss: 0.16020315513014793, Final Batch Loss: 0.042653556913137436
Epoch 240, Loss: 0.14563021808862686, Final Batch Loss: 0.022155694663524628
Epoch 241, Loss: 0.16684851795434952, Final Batch Loss: 0.02118539810180664
Epoch 242, Loss: 0.14582477509975433, Final Batch Loss: 0.03969934955239296
Epoch 243, Loss: 0.13185051549226046, Final Batch Loss: 0.03152872994542122
Epoch 244, Loss: 0.18772580288350582, Final Batch Loss: 0.05964372679591179
Epoch 245, Loss: 0.155909551307559, Final Batch Loss: 0.020080359652638435
Epoch 246, 

Epoch 344, Loss: 0.1309898979961872, Final Batch Loss: 0.050582513213157654
Epoch 345, Loss: 0.09255671221762896, Final Batch Loss: 0.008179026655852795
Epoch 346, Loss: 0.13067663926631212, Final Batch Loss: 0.061694297939538956
Epoch 347, Loss: 0.11888368893414736, Final Batch Loss: 0.010941524989902973
Epoch 348, Loss: 0.11565045919269323, Final Batch Loss: 0.020946471020579338
Epoch 349, Loss: 0.13248484022915363, Final Batch Loss: 0.03512432053685188
Epoch 350, Loss: 0.12304296530783176, Final Batch Loss: 0.03482505679130554
Epoch 351, Loss: 0.16752678249031305, Final Batch Loss: 0.09657741338014603
Epoch 352, Loss: 0.1185189695097506, Final Batch Loss: 0.040772695094347
Epoch 353, Loss: 0.09133201744407415, Final Batch Loss: 0.017920581623911858
Epoch 354, Loss: 0.11671847570687532, Final Batch Loss: 0.005875426344573498
Epoch 355, Loss: 0.10551252821460366, Final Batch Loss: 0.012390757910907269
Epoch 356, Loss: 0.08732648100703955, Final Batch Loss: 0.007749189622700214
Epoch 3

Epoch 453, Loss: 0.10503459768369794, Final Batch Loss: 0.045077044516801834
Epoch 454, Loss: 0.09114749170839787, Final Batch Loss: 0.010731926187872887
Epoch 455, Loss: 0.08390268729999661, Final Batch Loss: 0.026446759700775146
Epoch 456, Loss: 0.07074400968849659, Final Batch Loss: 0.017750512808561325
Epoch 457, Loss: 0.08453594893217087, Final Batch Loss: 0.013301641680300236
Epoch 458, Loss: 0.08966620080173016, Final Batch Loss: 0.013005937449634075
Epoch 459, Loss: 0.0591371594928205, Final Batch Loss: 0.00670918682590127
Epoch 460, Loss: 0.12448962335474789, Final Batch Loss: 0.08094365149736404
Epoch 461, Loss: 0.04598035989329219, Final Batch Loss: 0.003422951325774193
Epoch 462, Loss: 0.09073618613183498, Final Batch Loss: 0.025178158655762672
Epoch 463, Loss: 0.06473920261487365, Final Batch Loss: 0.017936725169420242
Epoch 464, Loss: 0.05245127156376839, Final Batch Loss: 0.011354340240359306
Epoch 465, Loss: 0.06865036394447088, Final Batch Loss: 0.004421261139214039
Ep

...

Epoch 4479, Loss: 0.0009408114667621703, Final Batch Loss: 7.84928761277115e-06
Epoch 4480, Loss: 0.0010953871478704968, Final Batch Loss: 3.515420394251123e-05
Epoch 4481, Loss: 0.00026857318312067946, Final Batch Loss: 6.17107143625617e-05
Epoch 4482, Loss: 0.00013370869237405714, Final Batch Loss: 1.550593697174918e-05
Epoch 4483, Loss: 0.0004524546839093091, Final Batch Loss: 5.484801658894867e-05
Epoch 4484, Loss: 9.677182379164151e-05, Final Batch Loss: 1.812089431041386e-05
Epoch 4485, Loss: 0.0006363990105455741, Final Batch Loss: 0.00010440043843118474
Epoch 4486, Loss: 0.0007625403441124945, Final Batch Loss: 2.5689578251331113e-06
Epoch 4487, Loss: 0.00018416788225295022, Final Batch Loss: 4.755991540150717e-05
Epoch 4488, Loss: 0.0006265769043238834, Final Batch Loss: 0.0001661132409935817
Epoch 4489, Loss: 0.00019717955001397058, Final Batch Loss: 7.247113535413519e-05
Epoch 4490, Loss: 0.0002284891090766905, Final Batch Loss: 0.00014781726349610835
Epoch 4491, Loss: 7.352

Epoch 4582, Loss: 0.00013276029494591057, Final Batch Loss: 3.852722147712484e-05
Epoch 4583, Loss: 0.00010856278640858363, Final Batch Loss: 3.235591429984197e-05
Epoch 4584, Loss: 8.67807439135504e-05, Final Batch Loss: 3.009668944287114e-05
Epoch 4585, Loss: 0.00045444603892974555, Final Batch Loss: 8.220593008445576e-05
Epoch 4586, Loss: 0.0013919630055170273, Final Batch Loss: 0.0002459383977111429
Epoch 4587, Loss: 0.006787686892494094, Final Batch Loss: 0.0003123761562164873
Epoch 4588, Loss: 0.0008064976918831235, Final Batch Loss: 0.00040420351433567703
Epoch 4589, Loss: 0.0007198869170679245, Final Batch Loss: 0.0003411920624785125
Epoch 4590, Loss: 0.02469338798618992, Final Batch Loss: 0.024386128410696983
Epoch 4591, Loss: 0.006951260002097115, Final Batch Loss: 0.002803178271278739
Epoch 4592, Loss: 0.0015460667345905676, Final Batch Loss: 0.0011794929159805179
Epoch 4593, Loss: 0.008624526693893131, Final Batch Loss: 0.00829935260117054
Epoch 4594, Loss: 0.00045913278700

Epoch 4688, Loss: 0.0005263700513751246, Final Batch Loss: 6.812715582782403e-05
Epoch 4689, Loss: 0.0005370488433982246, Final Batch Loss: 5.048359889769927e-05
Epoch 4690, Loss: 0.0017848576435426367, Final Batch Loss: 1.1647648534562904e-05
Epoch 4691, Loss: 0.004626162954082247, Final Batch Loss: 0.004266564268618822
Epoch 4692, Loss: 0.0009523183125565993, Final Batch Loss: 0.0006845167954452336
Epoch 4693, Loss: 0.0020203663625579793, Final Batch Loss: 0.000509000092279166
Epoch 4694, Loss: 0.0011002588653354906, Final Batch Loss: 1.8664206436369568e-05
Epoch 4695, Loss: 0.0012638818479899783, Final Batch Loss: 2.4668832338647917e-05
Epoch 4696, Loss: 0.0009008020788314752, Final Batch Loss: 0.00020030232553835958
Epoch 4697, Loss: 0.0009968052254407667, Final Batch Loss: 0.00046333958744071424
Epoch 4698, Loss: 0.0028369005231070332, Final Batch Loss: 0.0025537912733852863
Epoch 4699, Loss: 0.0006909946059749927, Final Batch Loss: 1.6605827113380656e-05
Epoch 4700, Loss: 0.00084

Epoch 4792, Loss: 0.0017293420469286502, Final Batch Loss: 0.001107138697989285
Epoch 4793, Loss: 0.00117914225847926, Final Batch Loss: 3.21576080750674e-05
Epoch 4794, Loss: 0.0022575750153919216, Final Batch Loss: 0.00011007965076714754
Epoch 4795, Loss: 0.06856362827238627, Final Batch Loss: 0.00032161184935830534
Epoch 4796, Loss: 0.0005107751348987222, Final Batch Loss: 0.00035245102480985224
Epoch 4797, Loss: 0.0026482506800675765, Final Batch Loss: 0.001205120817758143
Epoch 4798, Loss: 0.0035235286777606234, Final Batch Loss: 0.0033248511608690023
Epoch 4799, Loss: 0.03738733053614851, Final Batch Loss: 0.00011648815416265279
Epoch 4800, Loss: 0.000601532799009874, Final Batch Loss: 7.5900961746810935e-06
Epoch 4801, Loss: 0.0010224377474514768, Final Batch Loss: 0.0006429177010431886
Epoch 4802, Loss: 0.0012634130998776527, Final Batch Loss: 0.0007450981647707522
Epoch 4803, Loss: 0.0007911282427812694, Final Batch Loss: 9.324894563178532e-06
Epoch 4804, Loss: 0.0129862961202

Epoch 4899, Loss: 0.0008503049102728255, Final Batch Loss: 0.0006767807644791901
Epoch 4900, Loss: 0.0005553324444917962, Final Batch Loss: 6.05620771239046e-05
Epoch 4901, Loss: 0.0003117797641607467, Final Batch Loss: 5.456334838527255e-05
Epoch 4902, Loss: 0.0009984648859244771, Final Batch Loss: 0.0002771381114143878
Epoch 4903, Loss: 0.0004777208323503146, Final Batch Loss: 5.1428607548587024e-05
Epoch 4904, Loss: 0.00043354337503842544, Final Batch Loss: 0.00025355888647027314
Epoch 4905, Loss: 0.020752832657308318, Final Batch Loss: 0.0002709259570110589
Epoch 4906, Loss: 0.0007149205557652749, Final Batch Loss: 7.621783879585564e-07
Epoch 4907, Loss: 0.0008880015457179979, Final Batch Loss: 7.412615559587721e-06
Epoch 4908, Loss: 0.01599886869371403, Final Batch Loss: 0.000589317933190614
Epoch 4909, Loss: 0.00013110589316056576, Final Batch Loss: 7.208199531305581e-05
Epoch 4910, Loss: 0.0026458304237166885, Final Batch Loss: 1.7121141354436986e-05
Epoch 4911, Loss: 0.00035505

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[81  1  0]
 [ 0 65  0]
 [ 0  0 73]]
              precision    recall  f1-score   support

           0    1.00000   0.98780   0.99387        82
           1    0.98485   1.00000   0.99237        65
           2    1.00000   1.00000   1.00000        73

    accuracy                        0.99545       220
   macro avg    0.99495   0.99593   0.99541       220
weighted avg    0.99552   0.99545   0.99546       220



In [11]:
gen = Generator(z_dim = 110)
load_model(gen, "3 Label 7 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=110, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=33, bias=True)
    (4): Tanh()
  )
)

In [13]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 7)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [14]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[72  0  0]
 [ 0 72  0]
 [ 0  0 76]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        72
           1    1.00000   1.00000   1.00000        72
           2    1.00000   1.00000   1.00000        76

    accuracy                        1.00000       220
   macro avg    1.00000   1.00000   1.00000       220
weighted avg    1.00000   1.00000   1.00000       220



# User Classifier

In [15]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [16]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    elif y[k] == 8:
        y[k] = 4
    elif y[k] == 11:
        y[k] = 5
    else:
        y[k] = 6

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [18]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.894679546356201, Final Batch Loss: 1.9603559970855713
Epoch 2, Loss: 7.884167313575745, Final Batch Loss: 1.9705694913864136
Epoch 3, Loss: 7.868780970573425, Final Batch Loss: 1.9658170938491821
Epoch 4, Loss: 7.8607670068740845, Final Batch Loss: 1.9567447900772095
Epoch 5, Loss: 7.8448837995529175, Final Batch Loss: 1.9596424102783203
Epoch 6, Loss: 7.835121512413025, Final Batch Loss: 1.969307541847229
Epoch 7, Loss: 7.817938685417175, Final Batch Loss: 1.9432238340377808
Epoch 8, Loss: 7.81113600730896, Final Batch Loss: 1.9700939655303955
Epoch 9, Loss: 7.793146133422852, Final Batch Loss: 1.9553602933883667
Epoch 10, Loss: 7.751376390457153, Final Batch Loss: 1.922414779663086
Epoch 11, Loss: 7.7407472133636475, Final Batch Loss: 1.9402016401290894
Epoch 12, Loss: 7.721186876296997, Final Batch Loss: 1.9380511045455933
Epoch 13, Loss: 7.692253947257996, Final Batch Loss: 1.9323482513427734
Epoch 14, Loss: 7.645402908325195, Final Batch Loss: 1.9056884050369263
E

Epoch 120, Loss: 4.761829853057861, Final Batch Loss: 1.1650975942611694
Epoch 121, Loss: 4.720450282096863, Final Batch Loss: 1.2311360836029053
Epoch 122, Loss: 4.768608212471008, Final Batch Loss: 1.2076743841171265
Epoch 123, Loss: 4.627653360366821, Final Batch Loss: 1.0995984077453613
Epoch 124, Loss: 4.835886001586914, Final Batch Loss: 1.2386032342910767
Epoch 125, Loss: 4.746479868888855, Final Batch Loss: 1.1711076498031616
Epoch 126, Loss: 4.692871809005737, Final Batch Loss: 1.3063807487487793
Epoch 127, Loss: 4.638744950294495, Final Batch Loss: 1.1661179065704346
Epoch 128, Loss: 4.617733359336853, Final Batch Loss: 1.1089468002319336
Epoch 129, Loss: 4.543329238891602, Final Batch Loss: 1.0282597541809082
Epoch 130, Loss: 4.66612446308136, Final Batch Loss: 1.1977118253707886
Epoch 131, Loss: 4.675183296203613, Final Batch Loss: 1.153680682182312
Epoch 132, Loss: 4.587562441825867, Final Batch Loss: 1.0570013523101807
Epoch 133, Loss: 4.793037056922913, Final Batch Loss:

Epoch 236, Loss: 3.382176458835602, Final Batch Loss: 0.8651947975158691
Epoch 237, Loss: 3.4292789697647095, Final Batch Loss: 0.7854171991348267
Epoch 238, Loss: 3.4484748244285583, Final Batch Loss: 0.8241082429885864
Epoch 239, Loss: 3.346791386604309, Final Batch Loss: 0.7900749444961548
Epoch 240, Loss: 3.25363427400589, Final Batch Loss: 0.8025116324424744
Epoch 241, Loss: 3.249528169631958, Final Batch Loss: 0.7394524216651917
Epoch 242, Loss: 3.4468418955802917, Final Batch Loss: 0.8999666571617126
Epoch 243, Loss: 3.334760069847107, Final Batch Loss: 0.7846537828445435
Epoch 244, Loss: 3.2972081303596497, Final Batch Loss: 0.8107482194900513
Epoch 245, Loss: 3.4304621815681458, Final Batch Loss: 0.853219211101532
Epoch 246, Loss: 3.1606516242027283, Final Batch Loss: 0.7256135940551758
Epoch 247, Loss: 3.3860356211662292, Final Batch Loss: 0.8874524831771851
Epoch 248, Loss: 3.3434910774230957, Final Batch Loss: 0.8090512752532959
Epoch 249, Loss: 3.313083291053772, Final Bat

Epoch 355, Loss: 2.8371059894561768, Final Batch Loss: 0.6314424276351929
Epoch 356, Loss: 2.9289382696151733, Final Batch Loss: 0.7464593052864075
Epoch 357, Loss: 2.8546247482299805, Final Batch Loss: 0.752282977104187
Epoch 358, Loss: 2.830771028995514, Final Batch Loss: 0.6865600943565369
Epoch 359, Loss: 2.9037293195724487, Final Batch Loss: 0.740623414516449
Epoch 360, Loss: 2.892388880252838, Final Batch Loss: 0.7007708549499512
Epoch 361, Loss: 2.850001037120819, Final Batch Loss: 0.7431154847145081
Epoch 362, Loss: 2.9414098858833313, Final Batch Loss: 0.7018176317214966
Epoch 363, Loss: 2.7979738116264343, Final Batch Loss: 0.696622371673584
Epoch 364, Loss: 2.9103307127952576, Final Batch Loss: 0.7063626050949097
Epoch 365, Loss: 2.764580011367798, Final Batch Loss: 0.6654723286628723
Epoch 366, Loss: 2.918960452079773, Final Batch Loss: 0.7269576787948608
Epoch 367, Loss: 2.746744394302368, Final Batch Loss: 0.6782156229019165
Epoch 368, Loss: 2.9935149550437927, Final Batc

Epoch 470, Loss: 2.7283325791358948, Final Batch Loss: 0.7144932150840759
Epoch 471, Loss: 2.695101797580719, Final Batch Loss: 0.7419723868370056
Epoch 472, Loss: 2.528804838657379, Final Batch Loss: 0.5191831588745117
Epoch 473, Loss: 2.5064348578453064, Final Batch Loss: 0.6198750138282776
Epoch 474, Loss: 2.578514277935028, Final Batch Loss: 0.6518962979316711
Epoch 475, Loss: 2.6700241565704346, Final Batch Loss: 0.6974052786827087
Epoch 476, Loss: 2.677920639514923, Final Batch Loss: 0.6978408694267273
Epoch 477, Loss: 2.6498395204544067, Final Batch Loss: 0.6538849472999573
Epoch 478, Loss: 2.5224552750587463, Final Batch Loss: 0.574826717376709
Epoch 479, Loss: 2.639980673789978, Final Batch Loss: 0.6509215235710144
Epoch 480, Loss: 2.6358866095542908, Final Batch Loss: 0.6539267897605896
Epoch 481, Loss: 2.6178298592567444, Final Batch Loss: 0.5634018778800964
Epoch 482, Loss: 2.61919629573822, Final Batch Loss: 0.6101124882698059
Epoch 483, Loss: 2.485285997390747, Final Batc

...

Epoch 4529, Loss: 1.2277870327234268, Final Batch Loss: 0.29693618416786194
Epoch 4530, Loss: 1.2606996297836304, Final Batch Loss: 0.2418765127658844
Epoch 4531, Loss: 1.1462725698947906, Final Batch Loss: 0.247208833694458
Epoch 4532, Loss: 1.2078561186790466, Final Batch Loss: 0.29528650641441345
Epoch 4533, Loss: 1.239321231842041, Final Batch Loss: 0.3424593508243561
Epoch 4534, Loss: 1.1863926351070404, Final Batch Loss: 0.2781734764575958
Epoch 4535, Loss: 1.204152524471283, Final Batch Loss: 0.2806399464607239
Epoch 4536, Loss: 1.4624488651752472, Final Batch Loss: 0.5392325520515442
Epoch 4537, Loss: 1.3211407363414764, Final Batch Loss: 0.376618355512619
Epoch 4538, Loss: 1.1551049053668976, Final Batch Loss: 0.2975119948387146
Epoch 4539, Loss: 1.2563871145248413, Final Batch Loss: 0.3120400011539459
Epoch 4540, Loss: 1.1595192849636078, Final Batch Loss: 0.2876076102256775
Epoch 4541, Loss: 1.2729679942131042, Final Batch Loss: 0.32834377884864807
Epoch 4542, Loss: 1.262330

Epoch 4644, Loss: 1.266038566827774, Final Batch Loss: 0.28568992018699646
Epoch 4645, Loss: 1.2464888095855713, Final Batch Loss: 0.29927581548690796
Epoch 4646, Loss: 1.2427315413951874, Final Batch Loss: 0.29653990268707275
Epoch 4647, Loss: 1.1103334575891495, Final Batch Loss: 0.2240780144929886
Epoch 4648, Loss: 1.3522989153862, Final Batch Loss: 0.3002619743347168
Epoch 4649, Loss: 1.2502687275409698, Final Batch Loss: 0.3150210678577423
Epoch 4650, Loss: 1.1966864168643951, Final Batch Loss: 0.2830294966697693
Epoch 4651, Loss: 1.238889530301094, Final Batch Loss: 0.4088543951511383
Epoch 4652, Loss: 1.2310752868652344, Final Batch Loss: 0.30336347222328186
Epoch 4653, Loss: 1.3588300049304962, Final Batch Loss: 0.37964147329330444
Epoch 4654, Loss: 1.1597389876842499, Final Batch Loss: 0.25501763820648193
Epoch 4655, Loss: 1.0935272872447968, Final Batch Loss: 0.2457655370235443
Epoch 4656, Loss: 1.2057030647993088, Final Batch Loss: 0.2805196940898895
Epoch 4657, Loss: 1.1859

Epoch 4755, Loss: 1.29717355966568, Final Batch Loss: 0.35458213090896606
Epoch 4756, Loss: 1.0327169746160507, Final Batch Loss: 0.279802143573761
Epoch 4757, Loss: 1.2520713359117508, Final Batch Loss: 0.36272716522216797
Epoch 4758, Loss: 1.2827832996845245, Final Batch Loss: 0.315185010433197
Epoch 4759, Loss: 1.172174945473671, Final Batch Loss: 0.24855704605579376
Epoch 4760, Loss: 1.2639807611703873, Final Batch Loss: 0.2442997246980667
Epoch 4761, Loss: 1.2475461959838867, Final Batch Loss: 0.36794817447662354
Epoch 4762, Loss: 1.1879960596561432, Final Batch Loss: 0.27480649948120117
Epoch 4763, Loss: 1.12965689599514, Final Batch Loss: 0.2888336479663849
Epoch 4764, Loss: 1.314573496580124, Final Batch Loss: 0.4956381916999817
Epoch 4765, Loss: 1.0655092597007751, Final Batch Loss: 0.26162466406822205
Epoch 4766, Loss: 1.1204862594604492, Final Batch Loss: 0.2595764994621277
Epoch 4767, Loss: 1.301710605621338, Final Batch Loss: 0.29747575521469116
Epoch 4768, Loss: 1.4759064

Epoch 4873, Loss: 1.216128408908844, Final Batch Loss: 0.28981903195381165
Epoch 4874, Loss: 1.4278627038002014, Final Batch Loss: 0.3800791800022125
Epoch 4875, Loss: 1.5507558286190033, Final Batch Loss: 0.5120417475700378
Epoch 4876, Loss: 1.2342262268066406, Final Batch Loss: 0.3228306174278259
Epoch 4877, Loss: 1.2323959171772003, Final Batch Loss: 0.299152135848999
Epoch 4878, Loss: 1.3741221725940704, Final Batch Loss: 0.3378228545188904
Epoch 4879, Loss: 1.3215179145336151, Final Batch Loss: 0.35339289903640747
Epoch 4880, Loss: 1.2378810942173004, Final Batch Loss: 0.30015912652015686
Epoch 4881, Loss: 1.3328085839748383, Final Batch Loss: 0.29239514470100403
Epoch 4882, Loss: 1.2782068848609924, Final Batch Loss: 0.326535165309906
Epoch 4883, Loss: 1.2263135612010956, Final Batch Loss: 0.3144080638885498
Epoch 4884, Loss: 1.1864383220672607, Final Batch Loss: 0.29833173751831055
Epoch 4885, Loss: 1.0721582174301147, Final Batch Loss: 0.23937097191810608
Epoch 4886, Loss: 1.26

Epoch 4987, Loss: 1.197618007659912, Final Batch Loss: 0.29686328768730164
Epoch 4988, Loss: 1.2044507265090942, Final Batch Loss: 0.3033429980278015
Epoch 4989, Loss: 1.1407773047685623, Final Batch Loss: 0.24895624816417694
Epoch 4990, Loss: 1.2543337941169739, Final Batch Loss: 0.2688353955745697
Epoch 4991, Loss: 1.3983927369117737, Final Batch Loss: 0.5414756536483765
Epoch 4992, Loss: 1.1965278685092926, Final Batch Loss: 0.25854167342185974
Epoch 4993, Loss: 1.3044307678937912, Final Batch Loss: 0.34919828176498413
Epoch 4994, Loss: 1.2885797023773193, Final Batch Loss: 0.3830014765262604
Epoch 4995, Loss: 1.278082251548767, Final Batch Loss: 0.29426494240760803
Epoch 4996, Loss: 1.206554338335991, Final Batch Loss: 0.32202768325805664
Epoch 4997, Loss: 1.1780408322811127, Final Batch Loss: 0.2429296374320984
Epoch 4998, Loss: 1.2745114862918854, Final Batch Loss: 0.3301940858364105
Epoch 4999, Loss: 1.1091800928115845, Final Batch Loss: 0.25905612111091614
Epoch 5000, Loss: 1.2

In [19]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[49  0  0  0  6  0  0]
 [ 0 25  0  0  0  1  0]
 [ 0  1 23  0  0  0  0]
 [ 0  1  1 29  0  0  0]
 [ 2  2  0  0 21  0  1]
 [ 1  0  0  0  0 25  0]
 [ 0  5  0  2  0  0 25]]
              precision    recall  f1-score   support

           0    0.94231   0.89091   0.91589        55
           1    0.73529   0.96154   0.83333        26
           2    0.95833   0.95833   0.95833        24
           3    0.93548   0.93548   0.93548        31
           4    0.77778   0.80769   0.79245        26
           5    0.96154   0.96154   0.96154        26
           6    0.96154   0.78125   0.86207        32

    accuracy                        0.89545       220
   macro avg    0.89604   0.89954   0.89416       220
weighted avg    0.90425   0.89545   0.89650       220



In [20]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5, zero_division = 0))

[[19  0  0  0  0  0  0]
 [ 0 17  9  0  0  0  0]
 [ 0  0  0 16 14  0  0]
 [ 0  0  1 44  0  0  0]
 [ 0  2  0  0 33  0  0]
 [ 0  0  0  0  0 33  0]
 [ 0  0  0  0  0  0 32]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        19
           1    0.89474   0.65385   0.75556        26
           2    0.00000   0.00000   0.00000        30
           3    0.73333   0.97778   0.83810        45
           4    0.70213   0.94286   0.80488        35
           5    1.00000   1.00000   1.00000        33
           6    1.00000   1.00000   1.00000        32

    accuracy                        0.80909       220
   macro avg    0.76146   0.79635   0.77122       220
weighted avg    0.74926   0.80909   0.77059       220

